# Forming Groups

In [ ]:
""" Run on Google Colab """
!pip install rasterio
!pip install geopandas
!pip install geojson

In [ ]:
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
from rasterio.plot import show

import pandas as pd
import geopandas as gpd
import geojson
import json
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians, pi, isclose

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn import metrics

# Converting Mining Polygons to Geopandas file

### Ignore this section (need to done to add grId to mining_polygons) --- Already done

In [ ]:
with open("/content/drive/MyDrive/BTP/DataSet/mining_polygons.geojson") as f:
    gj = geojson.load(f)
features = gj['features']
for i in range(len(features)):
    features[i]['properties']['grId'] = -1
    features[i]['type'] = "Feature"
features[0]

In [ ]:
geojson_dict = '{"type": "FeatureCollection","crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },"features": []}'
geojson_obj = json.loads(geojson_dict)
geojson_obj['features'] = features
geojson_obj

In [ ]:
with open("/content/drive/MyDrive/BTP/DataSet/all_polygons_init.geojson", "w") as out_file:
    geojson.dump(geojson_obj,out_file)

In [ ]:
shape_path = "/content/drive/MyDrive/BTP/DataSet/all_polygons_init.geojson"
df = gpd.read_file(shape_path)
df

# Processing the init geojson to make group of mines

### Below code runs on indian_polygons_init which is same all_polygons_init\[all_polygons_init\['COUNTRY'] == "India"\]

In [ ]:
shape_path = "/content/drive/MyDrive/BTP/DataSet/indian_polygons_init.geojson"
df = gpd.read_file(shape_path)
df 

In [ ]:
#list of lat,lan
train = np.zeros((781,2))
for ind,row in df.iterrows():
    x,y = df.geometry[ind].exterior.coords.xy
    train[ind][0] = np.average(np.array(x))
    train[ind][1] = np.average(np.array(y))
print(train.shape)
train

In [ ]:
clustering=KMeans(n_clusters=170)
clustering.fit(train)
print(clustering.labels_)

In [ ]:
#new_df with new labels as per generated by kmean clustering
new_df = df.copy()
for i in range(781):
    new_df.at[i,'grId'] = clustering.labels_[i]
new_df

In [ ]:
# Saving new_df for debugging purposes (maintain same grId)
# new_df.to_file("/content/drive/MyDrive/BTP/DataSet/indian_polygon_with_grId.geojson", driver='GeoJSON')

In [ ]:
#loading new_df from saved file
new_df = gpd.read_file("/content/drive/MyDrive/BTP/DataSet/indian_polygon_with_grId.geojson")
new_df

In [ ]:
#curr in (longititude, latitude)
def give_polygon_boundary(avg,dx=5,dy=5):
    r_earth = 6373  #6378
    
    lat = avg[1]
    lon = avg[0]
    
    px = lat + (dy / r_earth) * (180 / pi)
    py = lon + (dx / r_earth) * (180 / pi) / cos(lat * pi/180)
    
    nx = lat - (dy / r_earth) * (180 / pi);
    ny = lon - (dx / r_earth) * (180 / pi) / cos(lat * pi/180)
    
    return [[ny,nx],[ny,px],[py,px],[py,nx],[ny,nx]]

def give_avg(grId,df):
    sum_x = 0
    sum_y = 0
    sum_len = 0
    for _,row in df[df['grId']==grId].iterrows():
        x,y = row['geometry'].exterior.coords.xy
        assert len(x)==len(y)
        sum_x += np.sum(np.array(x))
        sum_y += np.sum(np.array(y))
        sum_len += len(x)
    sum_x /= sum_len
    sum_y /= sum_len
    return sum_x,sum_y

"""
Ignore the below function (was earlier used to check whether the mining site is within 20x20 region of the group)
"""

def assertion(grId,df,dx=5,dy=5):
#     lon = avg_df[avg_df[2]==grId].iloc[0,0]
#     lat = avg_df[avg_df[2]==grId].iloc[0,1]
    lon,lat = give_avg(grId,df)
    lis = give_polygon_boundary((lon,lat),dx,dy)
    upX,loX,upY,loY = lis[2][1],lis[0][1],lis[2][0],lis[0][0]
    
    def check(pts,up,lo):
        pts = np.array(pts)
        return np.logical_and(pts>=lo,pts<=up).sum() == pts.size
        
    
    temp_df = df[df['grId']==grId]
    for _,row in temp_df.iterrows():
        lon_,lat_ = row['geometry'].exterior.coords.xy
        assert check(lon_,upY,loY) and check(lat_,upX,loX)
  
failed_assertion = []
group_ids = new_df['grId'].unique()
for grId in group_ids:
    try:
        assertion(grId,new_df,10,10)
    except:
        failed_assertion.append(grId)
print(len(failed_assertion))
print(failed_assertion)

# Making Square Shapes from Groups

### use save_sq_coords_v2 to generate boundary region for the group

In [ ]:
#Saving geometry as new csv (p1,p2,p3,p4,p5=p1)
def save_sq_coords_v1(df,dx=5,dy=5):
    group_ids = df['grId'].unique()
    master_lis = []
    for grId in group_ids:
        lon,lat = give_avg(grId,df)
        lis = give_polygon_boundary((lon,lat),dx,dy)
        lis.append([grId])
        master_lis.append(list(np.concatenate(lis).flat))
    arr = np.array(master_lis)
    np.savetxt("/content/drive/MyDrive/BTP/DataSet/sq_shapes_v1.csv",arr.reshape(arr.shape[0], -1) ,delimiter=',',
               header="p1y,p1x,p2y,p2x,p3y,p3x,p4y,p4x,p5y,p5x,grId",comments='')
    return

def save_sq_coords_v2(df,dx=5,dy=5):

    def give_polygon_boundary_v2(pos,grId):
        lon,lat = pos
        x,y = [],[]
        for _,row in df[df['grId']==grId].iterrows():
            x_,y_ = row.geometry.exterior.coords.xy
            x.extend(list(x_))
            y.extend(list(y_))
        min_lon,max_lon = min(x),max(x)
        min_lat,max_lat = min(y),max(y)
        #+/- 0.02 to both (+/- 1km)
        min_lon -= 0.02
        max_lon += 0.02
        min_lat -= 0.02
        max_lat += 0.02
        return [[min_lon,min_lat],[min_lon,max_lat],[max_lon,max_lat],[max_lon,min_lat],[min_lon,min_lat]]

    group_ids = df['grId'].unique()
    master_lis = []
    for grId in group_ids:
        lon,lat = give_avg(grId,df)
        lis = give_polygon_boundary_v2((lon,lat),grId)
        lis.append([grId])
        master_lis.append(list(np.concatenate(lis).flat))

    arr = np.array(master_lis)
    np.savetxt("/content/drive/MyDrive/BTP/DataSet/brazil_sq_shapes_v2.csv",arr.reshape(arr.shape[0], -1) ,delimiter=',',
               header="p1y,p1x,p2y,p2x,p3y,p3x,p4y,p4x,p5y,p5x,grId",comments='')
        

save_sq_coords_v2(new_df,10,10)

In [ ]:
#reading already saved shape_df
shape_df = pd.read_csv("/content/drive/MyDrive/BTP/DataSet/brazil_sq_shapes_v2.csv")
shape_df

# Making Image of Square Shapes

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)
  
dataset = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2019-01-01', '2019-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds);
# visualization = {
#     "min": 0,
#     "max": 255,
#     "bands": ['B1','B2', 'B3', 'B4','B5','B6', 'B7', 'B8','B8A','B9', 'B11', 'B12']
#     }

In [ ]:
shapes = []
group_ids = shape_df['grId'].unique()
print(group_ids)
for id in group_ids:
    temp_df = shape_df[shape_df['grId']==id]
    assert temp_df.shape[0]==1
    geometry = ee.Geometry.Polygon([[temp_df.iloc[0,0],temp_df.iloc[0,1]],
                                    [temp_df.iloc[0,2],temp_df.iloc[0,3]],
                                    [temp_df.iloc[0,4],temp_df.iloc[0,5]],
                                    [temp_df.iloc[0,6],temp_df.iloc[0,7]],
                                    [temp_df.iloc[0,8],temp_df.iloc[0,9]]])
    shapes.append(geometry)
shapes[1]

In [ ]:
# sentinel = dataset.median().clip(geometry)
clips = []
for shp in shapes:
    clip = dataset.median().clip(shp)
    clips.append(clip)

In [ ]:
# Uncomment to visualize
# Not required 
"""
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
# vis_params = {
#   'min': 0,
#   'max': 4000,
#   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)
my_map.add_ee_layer(sentinel, visualization, 'RGB');
# Display the map.
display(my_map)
"""

# Exporting Image

In [ ]:
tasks = []

#Making tasks
for i in range(len(clips)):
    task = ee.batch.Export.image.toDrive(
    image=clips[i].select(['B1','B2', 'B3', 'B4','B5','B6', 'B7', 'B8','B8A','B9', 'B11', 'B12']),
    description="BRA-"+str(int(group_ids[i])),   
    scale=10,
    region=shapes[i],
    folder='Brazil_Images',
    crs="EPSG:4326"
    )
    tasks.append(task)

In [ ]:
#Starting Tasks
for i in range(len(tasks)):
    tasks[i].start()

In [ ]:
#Task status
running = []
ready = tasks.copy()
completed = [] 

In [ ]:
for task in ready:
    if task.status()['state']=='COMPLETED':
        completed.append(task)
        ready.remove(task)
    elif task.status()['state']=='RUNNING':
        running.append(task)
        ready.remove(task)

for task in running:
    if task.status()['state']=='COMPLETED':
        completed.append(task)
        running.remove(task)

print("Current Running: ",len(running)," Completed: ",len(completed), " Yet to start: ",len(ready)," Total: ",len(tasks))

# Generating Masks of Images

In [ ]:
base_path = "/content/drive/MyDrive/BTP/Imagesv1/"

#do for each image/group
for img_name in os.listdir(base_path):
    img_name_ = os.path.splitext(img_name)[0]
    img_path = base_path + img_name
    
    with rasterio.open(img_path, "r") as src:
        raster_img = src.read()
        # rasterio.plot.show(src,cmap="pink")
        raster_meta = src.meta

    grId = int(img_name_.split('-')[1])

    #train_df generation from new_df
    train_df = new_df[new_df['grId']==grId]
    #Generate polygon
    def poly_from_utm(polygon, transform):
        poly_pts = []
        
        poly = cascaded_union(polygon)
        for i in np.array(poly.exterior.coords):
            
            # Convert polygons to the image CRS
            poly_pts.append(~transform * tuple(i))
            
        # Generate a polygon object
        new_poly = Polygon(poly_pts)
        return new_poly
    
    poly_shp = []
    im_size = (src.meta['height'], src.meta['width'])
    for num, row in train_df.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], src.meta['transform'])
            poly_shp.append(poly)
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append(poly)

    mask = rasterize(shapes=poly_shp,
                    out_shape=im_size)
    mask = mask*255
    # Plot the mask
    # plt.figure(figsize=(5,5))
    # plt.imshow(mask,cmap="gray")

    cv2.imwrite("/content/drive/MyDrive/BTP/Masksv1/"+img_name_+".png",mask)

# Visualizing RGB files from tif files

In [ ]:
base_path = "/content/drive/MyDrive/BTP/Imagesv1/"
image = clips[0].select(['B4', 'B3', 'B2'])

In [ ]:
imageRGB = image.visualize(['B5', 'B4', 'B3'], 0.5)
imageRGB